In [79]:
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import *
import scipy
import statsmodels.stats.multitest as smm

In [49]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [50]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [134]:
normal = data[data.Diagnosis == 'normal'].iloc[:, 2:]
early = data[data.Diagnosis == 'early neoplasia'].iloc[:, 2:]
cancer = data[data.Diagnosis == 'cancer'].iloc[:, 2:]

In [135]:
p_normal_early = np.array([scipy.stats.ttest_ind(normal[i],early[j], equal_var = False).pvalue for i,j in zip(normal, early)])

In [136]:
p_early_cancer = np.array([scipy.stats.ttest_ind(early[i],cancer[j],equal_var = False).pvalue for i,j in zip(early, cancer)])

In [137]:
common_ind = np.logical_or((p_normal_early < 0.05), (p_early_cancer < 0.05))

In [138]:
p_normal_early[p_normal_early < 0.05].shape

(1575,)

In [234]:
reject, p_corrected, a1, a2 = smm.multipletests(p_normal_early, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [235]:
normalavg = normal.mean(axis=0)
earlyavg = early.mean(axis=0)
canceravg = cancer.mean(axis=0)

In [236]:
def intbool(x,y):
    if(x>y):
        return x/y
    else:
        return -y/x
vintbool = np.vectorize(intbool)        

In [237]:
fc_normal_early = np.abs(vintbool(earlyavg, normalavg))

In [238]:
fc_normal_early[np.logical_and(p_corrected < 0.025, fc_normal_early > 1.5)]

array([ 1.66382607,  1.50978548,  1.75495105,  1.97486767])

In [239]:
reject, p_corrected, a1, a2 = smm.multipletests(p_early_cancer, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [240]:
fc_early_cancer = np.abs(vintbool(earlyavg, canceravg))

In [241]:
fc_early_cancer[np.logical_and(p_corrected < 0.025, fc_early_cancer > 1.5)].shape

(524,)

In [243]:
f = open('bio5.txt', 'w')
f.write(str(4))
f.close()

In [242]:
f = open('bio6.txt', 'w')
f.write(str(524))
f.close()

In [226]:
p_corrected[p_corrected < 0.025].shape

(79,)